In [1]:
from config import get_args

In [1]:
from itertools import cycle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler
import warnings 
warnings.filterwarnings("ignore", message="stft with return_complex=False is deprecated")

In [2]:
from dataset import*

In [3]:
config = get_args()
config

Namespace(debug=True, gpu_index='0', num_workers=8, verbose=1, save_path=None, train_datapath=PosixPath('/Data2/murmur/train'), test_datapath=PosixPath('/Data2/murmur/test'), external_datapath=PosixPath('/Data2/heart_sound_dataset'), external_data_subpath={'pysionet_sufhsdb': 'pysionet_sufhsdb', 'kaggle_set_a': 'kag_dataset_1/set_a', 'kaggle_set_b': 'kag_dataset_1/set_b'}, num_k=5, val_fold_num=4, sampling_rate=4000, window_length=0.05, hop_length=0.02, freq_high=800, freq_bins=40, train_seq_len=6, clean_noise=True, random_seed=0, max_epoch=250, train_bs=80, unlabel_bs=160, val_bs=120, learning_rate=0.0005, base_lr=4e-05, max_lr=0.001, final_lr=1e-05, training_patience=10, pos_weight=4.0, ema_factor=0.99, const_max=1, use_mixup=False, mixup_alpha=0.2, mixup_beta=0.2, mixup_label_type='soft')

In [3]:
data_folder = config.train_datapath
external_datapath = config.external_datapath
external_data_subpath = config.external_data_subpath
test_data_folder = config.test_datapath

num_k = config.num_k
val_fold_num = config.val_fold_num

split_ratio= 0.1 if config.debug else 1   
split_ratio

1

In [4]:
patient_df = load_patient_files(data_folder, stop_frac= split_ratio)
patient_df = create_folds(patient_df, num_k)
recording_df = patient_df_to_recording_df(patient_df)

recording_df_gq = recording_df[recording_df["patient_murmur_label"] != "Unknown"]
recording_df_bq = recording_df[recording_df["patient_murmur_label"] == "Unknown"]

train_recording_df = recording_df_gq[recording_df_gq['val_fold'] != val_fold_num]
val_recording_df = recording_df_gq[recording_df_gq["val_fold"] == val_fold_num]   

weaklabeled_df, unlabeled_df = get_external_df(external_datapath, config.external_data_subpath)
concat_unlabeld_df = pd.concat([weaklabeled_df, unlabeled_df], axis= 0).reset_index(drop=True)

test_patient_df = load_patient_files(test_data_folder, stop_frac= split_ratio)
test_recording_df = test_patient_df_to_recording_df(test_patient_df)   

In [5]:
#  data_folder, recording_paths, timings, murmur_labels, outcome_labels, 
#                  sampling_rate, window_length, window_step,
#                  clean_noise

train_dataset =  Stronglabeled_Dataset(data_folder, 
                            train_recording_df.index, train_recording_df.murmur_timing, 
                            train_recording_df.patient_murmur_label, train_recording_df.outcome_label,
                            sampling_rate= config.sampling_rate,
                            window_length= config.window_length,
                            window_step= config.hop_length,
                            clean_noise= config.clean_noise
                            )
unlabeled_dataset = Unlabeled_Dataset(external_datapath, 
                                    concat_unlabeld_df.mid_path, concat_unlabeld_df.filename, 
                                    config.sampling_rate, config.window_length, config.hop_length, config.freq_bins)

val_dataset = Stronglabeled_Dataset(data_folder, 
                            val_recording_df.index, val_recording_df.murmur_timing, 
                            val_recording_df.patient_murmur_label, val_recording_df.outcome_label,
                            sampling_rate= config.sampling_rate,
                            window_length= config.window_length,
                            window_step= config.hop_length,
                            clean_noise= config.clean_noise
                            )

In [6]:
len(train_dataset), len(unlabeled_dataset), len(val_dataset)

(1922, 626, 481)

In [9]:
sampler_unlabeled = RandomSampler(unlabeled_dataset, replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size= config.train_bs, shuffle=True, collate_fn= collate_fn, num_workers=config.num_workers)
unlabeled_dataloader = DataLoader(unlabeled_dataset, batch_size= config.unlabel_bs, sampler= sampler_unlabeled, collate_fn= collate_fn, num_workers=config.num_workers)
val_dataloader = DataLoader(val_dataset, batch_size= config.val_bs, shuffle=False,  collate_fn= collate_fn, num_workers=config.num_workers) 

for i, (batch_strong, batch_unlabel) in enumerate(zip(train_dataloader, cycle(unlabeled_dataloader))):
    
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [8]:
def collate_fn(x):
    # max_length = cal_max_frame_len(config.sampling_rate, config.sequence_length)
    # max_length= MAX_LENGTH
    max_length = 298
    FREQ_BINS = 40
    
    all_features = torch.zeros(len(x), FREQ_BINS, max_length)  # [bs, freq, frame]
    all_seq_labels = torch.ones(len(x), 3, max_length) * (-1) # S1, S2, Murmur, 
    all_murmur_labels = []
    all_outcome_labels = []
    pad_masks = []    
    all_filenames = []
    
    for idx, (features, seq_label, murmur_label, outcome_label, wav_len, filename) in enumerate(x):
        
        pad_mask = torch.ones(max_length)
        
        # 같거나 짧음
        if features.shape[-1] <= max_length:
            diff = max_length - features.shape[-1]
            start = random.randint(0, diff)
            end = start + features.shape[-1]
            all_features[idx, :, start : end] = features
            all_seq_labels[idx, : , start : end] = seq_label
            
            pad_mask[start:end] = 0.0 # 데이터 있는 부분이 0.0
            pad_masks.append(pad_mask)
            # actual_seq_lengths.append(len(seq_label))
            all_filenames.append(filename)
        # 더 길면
        else:
            diff = features.shape[-1] - max_length
            start = random.randint(0, diff)
            end =  start + max_length
            all_features[idx, :, :] = features[:, start : end]
            all_seq_labels[idx, :, :] = seq_label[:, start : end]
            
            pad_mask[:] = 0.0 # 모든 시퀀스에 데이터 있으므로 모두 0.0
            pad_masks.append(pad_mask)
            # actual_seq_lengths.append(max_length)

        all_murmur_labels.append(murmur_label)
        all_outcome_labels.append(outcome_label)
        all_filenames.append(filename)
        
    all_features = all_features.float()
    all_seq_labels = all_seq_labels.float()
    all_murmur_labels = torch.stack(all_murmur_labels).float()
    pad_masks = torch.stack(pad_masks).bool() #.float()

    return (all_features, all_seq_labels, pad_masks, all_murmur_labels, all_outcome_labels, all_filenames)

In [2]:
from model import MHA_LSTM_simpler

In [27]:
def get_act(act_func_name):
    if act_func_name.lower() == "relu":
        act_func = nn.ReLU()
    elif act_func_name.lower() == 'leakyrelu':
        act_func = nn.LeakyReLU()
    elif act_func_name.lower() == "gelu":
        act_func = nn.GELU()
    elif act_func_name.lower() == 'tanh':
        act_func = nn.Tanh()
    else:
        raise ValueError("you should add activation func in 'get_act function'")
    return act_func


# input_dim, hidden_dims, output_dim, act_func_name, dropout_rate, 

class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, act_func_name, dropout_rate):
        super(MLP, self).__init__()

        layers = nn.Sequential(nn.Linear(input_dim, hidden_dims[0]), 
                            nn.LayerNorm(hidden_dims[0]) , 
                            get_act(act_func_name),
                            nn.Dropout(dropout_rate),
                            )
        
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i + 1]))
            layers.append(get_act(act_func_name))
        
        layers.append(nn.Linear(hidden_dims[-1], output_dim))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [28]:
class MHA_LSTM_final(nn.Module):
    def __init__(self, model_params) -> None:
        super().__init__()
        
        self.rnn = nn.LSTM(**model_params["rnn_params"])
           
        self.selfattn_layer = nn.MultiheadAttention(**model_params["attn_module"]["MHA_params"])
        self.layer_norm1 = nn.LayerNorm(model_params["attn_module"]["layernorm_dim"])
        
        self.frame_linear = MLP(**model_params["frame_linear"])
        
        
        self.murmur_linear = nn.Sequential(
            nn.Linear(model_params["murmur_linear"]["input_dim"], model_params["murmur_linear"]["output_dim"]),
            )    
    
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.uniform_(m.weight, -0.1, 0.1)
                if m.bias is not None:
                    init.uniform_(m.bias, -0.1, 0.1)
            elif isinstance(m, nn.LSTM):
                for param in m.parameters():
                    if param.dim() >= 2:  # weight matrices
                        init.uniform_(param, -0.1, 0.1)
                    else:  # bias vectors
                        init.uniform_(param, -0.1, 0.1)
            elif isinstance(m, nn.MultiheadAttention):
                init.uniform_(m.in_proj_weight, -0.1, 0.1)
                if m.in_proj_bias is not None:
                    init.uniform_(m.in_proj_bias, -0.1, 0.1)
                init.uniform_(m.out_proj.weight, -0.1, 0.1)
                if m.out_proj.bias is not None:
                    init.uniform_(m.out_proj.bias, -0.1, 0.1)
        

    def forward(self, x, pad_mask= None):        
        x = x.permute(0, 2, 1) # [B, D, T] >> [B, T, D]
        
        # Bi-Rnn
        rnn_out, h_n = self.rnn(x)
        residual = rnn_out
        
        # Self MultiHeadAttention
        attn_output, attn_weights = self.selfattn_layer(rnn_out, rnn_out, rnn_out, key_padding_mask= pad_mask)

        attn_output += residual
        attn_output = self.layer_norm1(attn_output)
        
        seq_pred = self.frame_linear(attn_output).permute(0, 2, 1) # [B, T, C] >> [B, C, T]

        mm_linear_input = F.sigmoid(seq_pred).mean(dim=-1)[:, -1] # [B, C, T] >> [B, C] >> [B, Murmur]
        murmur_pred = self.murmur_linear(mm_linear_input.unsqueeze(-1).detach())
        return seq_pred, murmur_pred 

In [29]:
model_params = {}

model_params['rnn_params'] =  {"input_size": 40, 
                                "hidden_size": 60, 
                                "num_layers": 3, 
                                "batch_first": True, 
                                "bidirectional": True, 
                                "dropout": 0.1,
                                             }
                             
model_params["attn_module"] = {"MHA_params": {"embed_dim": model_params['rnn_params']["hidden_size"] * 2, 
                                                    "num_heads": 4, 
                                                    "batch_first": True, },
                               
                               "layernorm_dim": model_params['rnn_params']["hidden_size"] * 2}

model_params["frame_linear"] = {"input_dim": model_params['rnn_params']["hidden_size"] * 2, 
                                "hidden_dims": [40],
                                "output_dim": 3,
                                "act_func_name": 'gelu', 
                                "dropout_rate": 0.3,
                                }

model_params["murmur_linear"] = {"input_dim": 1, 
                                 "output_dim": 2}



In [30]:
model = MHA_LSTM_simpler(model_params)

In [31]:
print(model)

MHA_LSTM_simpler(
  (rnn): LSTM(40, 60, num_layers=3, batch_first=True, dropout=0.1, bidirectional=True)
  (selfattn_layer): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=120, out_features=120, bias=True)
  )
  (layer_norm1): LayerNorm((120,), eps=1e-05, elementwise_affine=True)
  (frame_linear): MLP(
    (model): Sequential(
      (0): Linear(in_features=120, out_features=40, bias=True)
      (1): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Dropout(p=0.3, inplace=False)
      (4): Linear(in_features=40, out_features=3, bias=True)
    )
  )
  (murmur_linear): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
  )
)
